In [1]:
import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [2]:
import os

cache_dir = "/root/autodl-fs"
os.environ["TRANSFORMERS_CACHE"] = cache_dir

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "JiazhenLiu01/llama3-8b-threeturn"
tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map={"": 0},
    torch_dtype='auto',
    #offload_folder="offload"
    cache_dir=cache_dir
).eval()

/root/miniconda3/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/671M [00:00<?, ?B/s]

In [ ]:
# # Read questions from question.txt file and extract them
# questions = []
# with open("question.txt", "r") as file:
#     lines = file.readlines()
#     for line in lines:
#         # Extract questions after the question number
#         question = line.split(".", 1)[-1].strip()
#         questions.append(question)

In [ ]:
# # Loop for dialogue inference
# with open("falcon_finetuned.txt", "a") as output_file:
#     for question in questions:
#         input_text = f"### Human: {question}### Assistant: "

#         # Encode input text into tokens
#         input_ids = tokenizer.encode(input_text, return_tensors="pt")
#         input_ids = input_ids.to("cuda")
#         # Use the trained model for dialogue inference
#         output = model.generate(input_ids, max_length=200, repetition_penalty=2.0)

#         # Decode the model output tokens into text
#         output_text = tokenizer.decode(output[0], skip_special_tokens=True)

#         response = output_text
#         # Find the position after "### Assistant:"
#         assistant_index = response.find("### Assistant:")

#         # Extract the response after "### Assistant:" until the last sentence with a period
#         assistant_response = response[assistant_index + len("### Assistant:"):]
#         last_period_index = assistant_response.rfind(".")
#         final_response = assistant_response[:last_period_index + 1]

#         print("Generated response:", final_response)

#         # Append each answer to falcon_finetuned.txt
#         output_file.write(final_response + "\n")

In [13]:
# input_text = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>Your name is Tom, and you are now communicating with a psychologist in a therapy session. Answer the following questions as Tom and respond in a casual tone. Each of your replies should be no longer than three sentences.<|eot_id|><|start_header_id|>### Human:<|end_header_id|>Do you feel anxiety?<|eot_id|><|start_header_id|>### Assistant:<|end_header_id|>"

# input_ids = tokenizer.encode(input_text, return_tensors="pt")
# input_ids = input_ids.to("cuda")

# output = model.generate(input_ids, max_length=200, repetition_penalty=1.2)

# output_text = tokenizer.decode(output[0], skip_special_tokens=True)

# response = output_text
# print("Generated response:", response)

# assistant_index = response.find("### Assistant:")

# assistant_response = response[assistant_index + len("### Assistant:"):]
# last_period_index = assistant_response.rfind(".")
# final_response = assistant_response[:last_period_index + 1]

# print("Generated response:",final_response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated response: systemYour name is Tom, and you are now communicating with a psychologist in a therapy session. Answer the following questions as Tom and respond in a casual tone. Each of your replies should be no longer than three sentences.### Human:Do you feel anxiety?### Assistant:I do still get very tired during the day. I can't stay awake all the time like I used to. I sort of will sit down on the couch for just 5 minutes, but it ends up going into an hour where I'm just lying there because I'm so tired. But that's been happening-- it's gotten a lot worse since football season ended. Injured in some way or another every single game. The last two games I wasn't even playing at all. Just coming onto the field, running around, jumping onto people when they were laying on the ground. All that sort of stuff was taking its toll, especially being hit by someone who has
Generated response: I do still get very tired during the day. I can't stay awake all the time like I used to. I sor

In [9]:
def extract_dialogue(text):
    tom_index = text.find("Tom:")
    if tom_index == -1:
        return None

    psychologist_index = min(text.find('Psychologist:',tom_index),text.find('Psychologists:',tom_index),text.find('Psychologist',tom_index),text.find('Psychologists',tom_index),
                             text.find('Psychiatrist:',tom_index),text.find('Psychiatrists:',tom_index),text.find('Psychiatrist',tom_index),text.find('Psychiatrists',tom_index),
                             text.find('psychologist:',tom_index),text.find('psychologists:',tom_index),text.find('psychologist',tom_index),text.find('psychologists',tom_index),
                             text.find('psychiatrist:',tom_index),text.find('psychiatrists:',tom_index),text.find('psychiatrist',tom_index),text.find('psychiatrists',tom_index),
                             text.find('\\n',tom_index),text.find('\n',tom_index))
                                       

    if psychologist_index != -1:
        dialogue = text[tom_index + len("Tom:"):psychologist_index]
    else:
        dialogue = text[tom_index + len("Tom:"):]

    last_period_index = max(dialogue.rfind('.'),dialogue.rfind('--'),dialogue.rfind('?'))

    dialogue = dialogue[:last_period_index+1]
    
    return dialogue.strip() 

In [10]:
def infer(content):
        input_text = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>Pentending Your name is Tom, and you are now communicating with a psychologist in a therapy session. Respond to the psychologist as Tom using English words and respond in a casual tone. Your reply should be no longer than one sentence. <|eot_id|><|start_header_id|> Psychologist: <|end_header_id|>{content}<|eot_id|><|start_header_id|> Tom: <|end_header_id|>"
        # Encode input text into tokens
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        input_ids = input_ids.to("cuda")
        # Use the trained model for dialogue inference
        output = model.generate(input_ids, repetition_penalty=1.5, pad_token_id=tokenizer.eos_token_id,max_new_tokens=50)

        # Decode the model output tokens into text
        output_text = tokenizer.decode(output[0], skip_special_tokens=True)

        output_text = output_text.splitlines()[0]
        
        response = extract_dialogue(output_text)

        print("Generated response:", response)

        # Append each answer to falcon_finetuned.txt
        return response

In [13]:
#convert txt to json
def append_data(text):
    data = {
    "prompt": [],
    "chosen": [],
    "rejected": [],}
    current_content = []
    current_role = None
    current_messages = ""
    lines = text.split('\n') 
    
    for i, line in enumerate(lines):     
        if i + 2 < len(lines):
            if line.strip() == '':
                continue
            parts = line.strip().split(' ', 2)
            role = parts[0]
            content = parts[-1]

            if role == "S1:" or role == "S1":
                data["prompt"].append("### Human: "+ content)
                print("prompt:",content)
                data["rejected"].append("### Assistant: ")
                # data["rejected"].append("### Assistant: "+infer(content))
                parts = lines[i + 2].strip().split(' ', 2)
                role = parts[0]
                content = parts[-1]
                data["chosen"].append("### Assistant: "+ content)
                print("chosen:",content)                
                current_messages = ""
    return data

In [14]:
from dataclasses import dataclass, field
from typing import Optional
from huggingface_hub import HfApi
from datasets import Dataset



folder_name = "cleared_data" 
text = "" 

for filename in os.listdir(folder_name):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_name, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            text += file.read()

data = append_data(text)
example_dataset = Dataset.from_dict(data)
example_dataset.push_to_hub("JiazhenLiu01/dpo_llama3_362_threeturn_empty_neg",token = "xxxxxx")


prompt: Okay.
chosen: Cool.
prompt: All gone. So it's Mom's this week.
chosen: Yep.
prompt: Yep. And then Dad's next week.
chosen: And then Dad's next week. So we just came back from Dad's on the Monday after--
prompt: From school.
chosen: Yeah. Monday afternoon.
prompt: I think using this is going to help me figure out which week we're on.
chosen: Yeah. So we are on--
prompt: So I brought it in so we can--
chosen: So we just come off--
prompt: Wednesday. Is that--?
chosen: Because it's--
prompt: Or is it a bit different?
chosen: Because it's holidays, we have a week with Dad, well, around about. It's not a straight week with Mom, a straight week with Dad. But I think we came off--
prompt: It was--
chosen: No. We came off Dad's week. So Dad had us from the Friday after school.
prompt: Because this was week one last week when we saw each other.
chosen: Yeah. So--
prompt: We were on here, yep.
chosen: Yeah. We had Dad on the Friday, Saturday, Sunday, Monday afternoon, and then we went ba

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/JiazhenLiu01/dpo_llama3_362_threeturn_empty_neg/commit/344cf4ac3f13c2e647e3187c15a0b756c0b760da', commit_message='Upload dataset', commit_description='', oid='344cf4ac3f13c2e647e3187c15a0b756c0b760da', pr_url=None, pr_revision=None, pr_num=None)